In [ ]:
import ee

# Initialize Earth Engine API
ee.Initialize()

# Define AOI as a polygon
AOI = ee.Geometry.Polygon(
    [[[77.40600099670314, 23.250822912946425],
      [77.40806093322658, 23.249403425472668],
      [77.41072168456935, 23.252281814882856],
      [77.4090909014883, 23.253464696627372]]])

# Create a feature collection from AOI
featureCollection = ee.FeatureCollection([AOI])

# Define geometry as a point
geom = ee.Geometry.Point([77.40600099670314, 23.250822912946425])

# Define a function to mask clouds in Sentinel-2 images
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000).copyProperties(image,['system:time_start'])

# Define a function to add dissolved oxygen bands to Sentinel-2 images
def addDissolvedOxygen(image):
    return image.addBands(ee.Image(-0.0167).multiply(image.select('B8')) \
        .add(ee.Image(0.0067).multiply(image.select('B9'))) \
        .add(ee.Image(0.0083).multiply(image.select('B11'))) \
        .add(ee.Image(9.577)).rename('dissolvedOxygen'))

# Load Sentinel-2 surface reflectance data
s2 = ee.ImageCollection("COPERNICUS/S2_SR")

# Define start and end dates
startDate = ee.Date.fromYMD(2013, 1, 1)
endDate = ee.Date.fromYMD(2022, 12, 31)

# Filter the Sentinel-2 image collection
originalCollection = s2.filter(ee.Filter.date(startDate, endDate)) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
    .filter(ee.Filter.bounds(geom)) \
    .map(maskS2clouds) \
    .map(addDissolvedOxygen)

# Select the dissolved oxygen band and extract a single image
collection = originalCollection.select('dissolvedOxygen')
image = originalCollection.first()

# Define a function to create a feature with dissolved oxygen value and date
def feature(image):
    date = image.date().format('yyyy-MM-dd')
    value = image.clip(geom) \
        .reduceRegion(reducer=ee.Reducer.mean(), scale=30) \
        .get('dissolvedOxygen')
    return ee.Feature(None, {'value': value, 'date': date})

# Map the feature function over the dissolved oxygen band and create a time series
timeSeries = collection.map(feature)

# Convert the time series to a table and save as a CSV file
table = ee.batch.Export.table.toDrive(collection=timeSeries, description='dissolved_oxygen', 
                                      folder='GEE Output', fileNamePrefix='dissolved_oxygen', fileFormat='CSV')
table.start()
